**<h1>Exemplo: Curvas de Aprendizado (Learning Curves) + Métricas de Avaliação</h1>**
<h2>Classificador de formas geométricas com três classes: "círculos", "quadrados" e "triângulos"</h2>
<h4>Exemplo contém uso de callbacks de treino (modelcheckpoint, earlystopping)</h4>
<hr>
<h3>Prof. Dr. Rafael Rieder</h3>
<h5>Universidade de Passo Fundo. Última atualização: 10/10/2023</h5>

Importação das dependências

In [ ]:
import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
import numpy as np
from keras.layers import Conv2D
from keras.layers import Reshape
from keras.layers import MaxPooling2D
from keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import load_img, img_to_array
from keras.regularizers import l2
from sklearn.preprocessing import label_binarize
from itertools import cycle
from matplotlib import pyplot as plt
import pandas as pd
# from keras.models import load_model

Montagem do Drive e definição dos caminhos

In [ ]:
# CURRDIR = os.path.dirname(__file__)
CURRDIR = f"/content/drive/MyDrive/Colab Notebooks/ML class/"
TRAINPATH = os.path.join(CURRDIR, "train/")
VALPATH = os.path.join(CURRDIR, "val/")
TESTPATH = os.path.join(CURRDIR, "test/")
MODELFILEPATH = os.path.join(CURRDIR, "save/weights.keras") # novo formato de arquivo (hdf5 deprecated)

<h1>Construção e treinamento do modelo CNN</h1>

Define qual a posição do canal de cores num possível array de leitura via biblioteca de Processamento de Imagens (channels_last (rows, cols, channels) // channels_first (channels, rows, cols)

In [ ]:
K.set_image_data_format('channels_last')

<h3>Montagem da CNN</h3>

Informações:

**Ativação Relu:** se o valor 'x' que estiver na saída do neurônio for x < 0, então x = 0; senão (positivo) é o próprio valor (x = x). É uma função computacionalmente leve, entretanto não é centrada em zero. Tende a propagar valores positivos para manter neurônios relevantes ativados.

**Ativação Softmax:** converte um vetor de values para uma distribuição de probabilidades. Ou seja, define um percentual de certeza de acerto para cada classe.

**Otimizador Adam:** é um método de gradiente descendente estocástico baseado na estimativa de momento adaptável de primeira e segunda ordem.
O método é realmente eficiente ao trabalhar com grandes problemas envolvendo muitos dados ou parâmetros. Requer menos memória e é eficiente.

Outros ativadores podem ser vistos em: https://keras.io/api/layers/activations/

Outros otimizadores podem ser vistos em: https://keras.io/api/optimizers/

Mais info sobre modelos Sequential e Functional podem ser vistos em: https://www.tensorflow.org/guide/keras/sequential_model e https://www.tensorflow.org/guide/keras/functional

In [ ]:


model = Sequential()  # define que o modelo é composto de camadas sequenciais
# Primeiras duas camadas (aguardam imagens de um canal, i.e. grayscale)
# Se quiser analisar os 3 canais de cores, informar na construção do modelo: (w, h, 3) <==> (3, w, h)
model.add(Reshape((1, 100, 100), input_shape=(100, 100, 1)))  # define que todas as entradas serão tratadas como imagens 100x100, 1 canal
model.add(Conv2D(32, (5, 5), input_shape=(1, 100, 100), activation='relu', padding='same'))  # define o uso de 24 filtros 5x5, função de ativação Relu

# Exemplo da camada 5x5 com regularização aplicada
# model.add(Conv2D(32, (5, 5), input_shape=(1, 100, 100), activation='relu', padding='same', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))

# Demais camadas
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))  # define um downsize das imagens (50% para cada dimensão)
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))  # define o uso de 12 filtros 5x5, função de ativação Relu
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))  # define um downsize das imagens (50% para cada dimensão)
model.add(Flatten())  # transforma a matriz 2D em um único vetor
model.add(Dense(64, activation='relu'))    # conecta todas as saídas do vetor a uma rede FC de 72 neurônios, ativação Relu
model.add(Dense(3, activation='softmax'))  # conecta os 64 neurônios da camada anterior a 3 neurônios (classes possíveis de saída), ativação Softmax (define a % de certeza)

# Compilando o modelo
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # categorial_crossentropy serve para várias classes de saída (se não, seria binary_crossentropy)

# Adicionando mais métricas na compilação, para monitoramento com learning curves
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy", tf.keras.metrics.Precision(name="precision"), tf.keras.metrics.Recall(name="recall"), tf.keras.metrics.AUC(name="AUC")])  # categorial_crossentropy serve para várias classes de saída (se não, seria binary_crossentropy)

# Incluindo uma métrica customizada
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["get_F1_score", "accuracy", tf.keras.metrics.Precision(name="precision"), tf.keras.metrics.Recall(name="recall"), tf.keras.metrics.AUC(name="AUC")])  # categorial_crossentropy serve para várias classes de saída (se não, seria binary_crossentropy)


Sumarização do modelo CNN gerado

In [ ]:
model.summary()

Preparando o dataset de treinamento. Nesse caso, estamos normalizando as imagens e definindo uma configuração de aumentação de dados (no caso, além da imagem original, uma versão com flip horizontal pode ser considerada na rodada de treinamento).

ADENDO: Fiz a inclusão de um dataset para validação usando model.evaluation, e aumentei proporcionalmente os datasets.

Divisão do dataset organizada em diretórios de 60/20/20. São 30 imagens por classe para treinamento, e 10 imagens por classe para validação. Mais adiante, para teste, são 10 por classe.

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
    horizontal_flip=True)

X_train = train_datagen.flow_from_directory(
    TRAINPATH,
    target_size=(100, 100),
    batch_size=10,
    color_mode='grayscale')  # se colorido, 'rgb' (lembre que o modelo precisa estar preparado também para 3 canais)

val_datagen = ImageDataGenerator(rescale=1./255)

X_val = val_datagen.flow_from_directory(
    VALPATH,
    target_size=(100, 100),
    color_mode='grayscale')  # se colorido, 'rgb' (lembre que o modelo precisa estar preparado também para 3 canais)

Y_train = X_train.classes
Y_val = X_val.classes

#X_train.class_indices
#Y_train.shape,Y_val.shape

# **Curvas de Aprendizado**
Criação de uma função para desenhar os gráficos.
Importante: só podemos plotar gráficos de métricas que estamos monitorando no modelo em compilação. Verificar definição de model.compile(...).

In [ ]:
# Métrica F1-Score customizada para monitoramento de modelo em compilação
# Por padrão, ela não está disponível por TF/keras. Porém, podemos criar métricas
# personalizadas e adicionar ao processo.
def get_F1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

# Geração das curvas
def plotar_historico(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Treinamento', 'Validação'], loc='upper left'),
    plt.show()
    #Caso queira imprimir o gráfico, ao invés de mostrar, use:
    #plt.savefig(os.path.join(CURRDIR, "save/model_loss.png"))
    print('')

    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Treinamento', 'Validação'], loc='upper left'),
    plt.show()
    print('')

    plt.plot(history.history['precision'])
    plt.plot(history.history['val_precision'])
    plt.title('model precision')
    plt.ylabel('precision')
    plt.xlabel('epoch')
    plt.legend(['Treinamento', 'Validação'], loc='upper left')
    plt.show()
    print('')

    plt.plot(history.history['AUC'])
    plt.plot(history.history['val_AUC'])
    plt.title('model AUC')
    plt.ylabel('AUC')
    plt.xlabel('epoch')
    plt.legend(['Treinamento', 'Validação'], loc='upper left')
    plt.show()
    print('')

    plt.plot(history.history['recall'])
    plt.plot(history.history['val_recall'])
    plt.title('model recall')
    plt.ylabel('recall')
    plt.xlabel('epoch')
    plt.legend(['Treinamento', 'Validação'], loc='upper left')
    plt.show()
    print('')
'''
    plt.plot(history.history['get_F1_score'])
    plt.plot(history.history['val_get_F1_score'])
    plt.title('Model F1 score')
    plt.ylabel('F1 score')
    plt.xlabel('Epoch')
    plt.legend(['Treinamento', 'Validação'], loc='upper left')
    plt.show()
    print('')
'''

Definindo que queremos salvar o modelo com seus pesos (a ideia é depois usar somente o arquivo final com o modelo inteligente)

ADENDO: inclui uma callback early_stop para parar o treinamento antes de tender a um overfitting.

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    mode='min',
    verbose=1,
    patience=5)
#Early stopping to avoid overfitting of model

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=MODELFILEPATH,
    monitor='loss',
    verbose=1,
    mode='max')
#Save the Keras model or model weights at some frequency

callbacks_list = [checkpoint, early_stop]

Treinando o modelo para gerar, após as épocas, o modelo definitivo.
ADENDO: aumentei o número de épocas (20 para 50) e o batch_size (5 para 10).

In [ ]:
history = model.fit(
    X_train,
    steps_per_epoch=len(X_train),
    validation_data= X_val,
    validation_steps=len(X_val),
    epochs=50,
    batch_size=10,
    verbose=1,
    callbacks=callbacks_list)

ADENDO: como estou usando um dataset de validação, faço um evaluate nele.
Importante que as métricas precisam ser parecidas com o treinamento. Quanto mais próximo, melhor o ajuste.

In [ ]:
prediction = model.evaluate(X_val, batch_size=10, verbose=1)
dict(zip(model.metrics_names, prediction))

# ** Desenhando as curvas de aprendizado, e salvando o histórico de compilação **

In [ ]:
# plotar historico
plotar_historico(history)

# salvar historico
hist_df = pd.DataFrame(history.history)
hist_df.to_csv(os.path.join(CURRDIR, "save/history.csv"))

<h1>Usando o modelo e testando sua predição com novas entradas</h1>

Carregando o arquivo que contém o modelo treinado (apenas um exemplo de como carregar um modelo já criado - não usado nesse exemplo).

In [ ]:
# model = load_model(MODELFILEPATH)
# model = load_model(MODELFILEPATH, custom_objects={'get_F1_score':get_F1_score}) # <<== esta linha aqui se salvou alguma métrica customizada

Lendo uma nova imagem e preparando ela para verificar a qual classe pertence (no caso, círculo, quadrado ou triângulo)

ADENDO: para o exercício, estou fazendo uma varredura no diretório usando sorted (para buscar em ordem alfabética)

Importante que a entrada precisa ser convertida para a mesma configuração que o modelo foi treinado (nesse caso, 100x100, grayscale)

-----

Faz a predição da entrada, e pega somente a classe que tem o maior percentual de certeza

ADENDO: faço também uma rápida predição aqui, arquivo por arquivo.

Saída final de categorização

In [ ]:
#img = image.load_img(FILETESTPATH, target_size=(100, 100), color_mode='grayscale')
#y = image.img_to_array(img)
#y = np.expand_dims(y, axis=0)

# load all images into a list
images = []
for img in sorted(os.listdir(TESTPATH)):
  print(img)
  img = os.path.join(TESTPATH, img)
  img = image.load_img(img, target_size=(100, 100), color_mode='grayscale')  # se colorido, 'rgb'
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  images.append(img)
  predict = model.predict(img)
  classes = np.argmax(predict, axis=1)
  if(classes[0]==0): print("CÍRCULO!")
  elif(classes[0]==1): print("QUADRADO!")
  else: print("TRIÂNGULO!")
  print("----------")

ADENDO: Aqui estou lendo o banco de imagens. Como as imagens já estão em tons de cinza, deixei comentado a divisão.

Carrega para uma pilha, e faz a predição de todas de uma só vez!

In [ ]:
# stack up images list to pass for prediction
#images = images/255
images = np.vstack(images)
result = model.predict(images)

ADENDO: Cálculo das métricas aqui. Um pequeno ajuste para Roc AUC Score quando usamos 3 ou mais classes. Deixei comentado o padrão para casos binários (2 classes).

In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt

Y_true = np.array([1,1,1,2,0,2,2,1,1,0,0,0,0,0,1,2,0,0,0,2,1,1,2,1,2,2,1,2,2,0]) # Como está no diretório cada img (Segue a ordem alfabética)
Y_pred = np.argmax(result, axis=1) # Pega só o maior valor, torna a matriz 1D

cm = confusion_matrix(Y_true, Y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Círculo", "Quadrado", "Triângulo"])
disp.plot(cmap='GnBu')

plt.show()
# Por padrão, average = binary (2 classes). Aqui são 3, optei em usar "micro".
print('Precision: %.3f' % precision_score(Y_true, Y_pred, average='micro'))
print('Recall: %.3f' % recall_score(Y_true, Y_pred, average='micro'))
print('F1: %.3f' % f1_score(Y_true, Y_pred, average='micro'))
print('Accuracy: %.3f' % accuracy_score(Y_true, Y_pred))

# Para duas classes:
# print('Roc AUC Score: %.3f' % roc_auc_score(Y_true, Y_pred, average='macro'))
# fpr, tpr, thresholds = roc_curve(Y_true, Y_pred)

# plt.xlabel('Taxa de Falsos Positivos')
# plt.ylabel('Taxa de Verdadeiros Positivos')
# plt.title('Curva ROC')
# plt.plot(fpr, tpr)

class_names = ['Círculo', 'Quadrado', 'Triângulo']

# Binarize ytest with shape (n_samples, n_classes)
ytrues = label_binarize(Y_true, classes=[0, 1, 2])
n_classes = ytrues.shape[1]

# Binarize ypreds with shape (n_samples, n_classes)
ypreds = label_binarize(Y_pred, classes=[0, 1, 2])

print('Roc AUC Score (OVR): %.3f' % roc_auc_score(ytrues, ypreds, multi_class='ovr'))

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
  fpr[i], tpr[i], _ = roc_curve(ytrues[:, i], ypreds[:, i])
  roc_auc[i] = auc(fpr[i], tpr[i])
colors = cycle(['blue', 'red', 'green'])
for i, color in zip(range(n_classes), colors):
  plt.plot(fpr[i], tpr[i], color=color, lw=1.5,
            label='ROC curve of class {0} (area = {1:0.2f})'
            ''.format(class_names[i], roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw=1.5)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for multi-class data')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score

# Tratamento para Curva PR Multiclasse
print('')
precision = dict()
recall = dict()
average_precision = dict()
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(ytrues[:, i], ypreds[:, i])
    average_precision[i] = average_precision_score(ytrues[:, i], ypreds[:, i])

# A "micro-average": quantifying score on all classes jointly
precision["micro"], recall["micro"], _ = precision_recall_curve(ytrues.ravel(), ypreds.ravel())
average_precision["micro"] = average_precision_score(ytrues, ypreds, average="micro")

colors = cycle(['blue', 'red', 'green'])
for i, color in zip(range(n_classes), colors):
  plt.plot(recall[i], precision[i], color=color, lw=1.5,
            label='PR curve of class {0} (area = {1:0.2f})'
            ''.format(class_names[i], average_precision[i]))
plt.plot([0, 1], [0, 1], 'k--', lw=1.5)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Prediction')
plt.title('Precision-Recall curve to multi-class')
plt.legend(loc="lower right")
plt.show()

print('AP Score (Micro)...: {0:0.3f}'.format(average_precision["micro"]))


In [ ]:
#Fontes para Model Evaluation:
#https://www.tensorflow.org/guide/keras/train_and_evaluate
#https://www.analyticsvidhya.com/blog/2021/07/step-by-step-guide-for-image-classification-on-custom-datasets/
#https://machinelearningmastery.com/how-to-calculate-precision-recall-f1-and-more-for-deep-learning-models/
#https://www.projectpro.io/recipes/evaluate-keras-model#mcetoc_1g2a1msp0d
#https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
#https://gist.github.com/ritiek/5fa903f97eb6487794077cf3a10f4d3e
#https://androidkt.com/how-to-predict-images-using-trained-keras-model/
#https://stackoverflow.com/questions/33547965/computing-auc-and-roc-curve-from-multi-class-data-in-scikit-learn-sklearn
#https://stackoverflow.com/questions/63303682/sklearn-multiclass-roc-auc-score
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html

#Fontes para Bias/Variance/Learning Curves:
#https://www.tensorflow.org/tutorials/keras/overfit_and_underfit?hl=pt-br
#https://www.geeksforgeeks.org/bias-vs-variance-in-machine-learning/
#https://rasbt.github.io/mlxtend/user_guide/evaluate/bias_variance_decomp/
#https://www.kaggle.com/code/azminetoushikwasi/mastering-bias-variance-tradeoff
#https://www.kaggle.com/code/nicolaugoncalves/curvas-de-aprendizado
#https://bixtecnologia.com.br/como-entender-vies-e-variancia-em-modelos-preditivos/
#https://stackabuse.com/the-bias-variance-trade-off-in-machine-learning/
#https://medium.com/data-hackers/o-que-é-bias-variance-tradeoff-a5bc19866e4b
#https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/